<!-- # Processing the final dataset
 -->


# Export data as CSV

In [1]:
import ee
import geemap
from gee_0_utils import *

initialize()
config = ProjectConfig()
roi = config.roi
data_folder = config.data_folder
last_year = config.last_year


## Main Model Dataset

### Age, Biomass

In [2]:
# Fire and Land Use
age = ee.Image(f"{data_folder}/mapbiomas_{last_year}")

ESA_CCI = ee.Image(f"projects/sat-io/open-datasets/ESA/ESA_CCI_AGB/CCI_BIOMASS_100m_AGB_{last_year}_v51").select("AGB").rename("biomass")

fire = (ee.Image("projects/mapbiomas-public/assets/brazil/fire/collection3/mapbiomas_fire_collection3_annual_burned_coverage_v1")
    .select([f"burned_coverage_{year}" for year in config.range_1985_2020])
    .byte()
    .rename([str(year) for year in config.range_1985_2020])
    .gt(0)
    .reduce('sum').rename("num_fires")).unmask(0)

floodable_forests = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
        .select(f"classification_{last_year}").eq(6)).rename("floodable_forests")

secondary_area = ee.Image(f"{data_folder}/secondary_area")
pasture_area = ee.Image(f"{data_folder}/pasture_area")


In [5]:
reduceRegions_10k = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grids/reduceRegions_10k")
reduceRegions_1k = ee.FeatureCollection("projects/amazon-forest-regrowth/assets/grids/reduceRegions_1k")

map = geemap.Map()
map.addLayer(age, {}, "age")
map.addLayer(reduceRegions_10k, {}, "reduceRegions_10k")
map.addLayer(reduceRegions_1k, {}, "reduceRegions_1k")
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [7]:
clip_area = map.draw_layer()

map.addLayer(clip_area, {}, "clip_area")

TypeError: 'EELeafletTileLayer' object is not callable

### Surrounding Landscape

In [3]:
quarters_ecoreg_biomass = ee.Image("projects/amazon-forest-regrowth/assets/quarters_ecoreg_biomass")
ecoreg = ee.Image("projects/amazon-forest-regrowth/assets/ecoreg")
distance_deep_forest = ee.Image(f"{data_folder}/distance_deep_forest").rename("dist")
sur_cover = ee.Image(f"{data_folder}/sur_cover")

distance_gt_1000 = distance_deep_forest.gt(1000).selfMask()
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_10k = ee.Image(f"{data_folder}/mature_biomass_10k")
nearest_mature = ee.Image(f"{data_folder}/nearest_mature")

### Environmental

In [4]:
categorical = ee.Image(f"{data_folder}/categorical")

topography = ee.Image("CSP/ERGo/1_0/Global/ALOS_landforms").rename("topography") # 90m resolution

soil = ee.Image(f"{data_folder}/soilgrids")

terraclim = ee.Image(f"{data_folder}/terraclim_1958_2019")

### Export CSV


In [5]:
# Define reprojection function
def reproject_mean(img):
    return img.reduceResolution(ee.Reducer.mean(), maxPixels=65536).reproject(age.projection())

def reproject_first(img):
    return img.reduceResolution(ee.Reducer.first(), maxPixels=65536).reproject(age.projection())

# Reproject continuous variables using mean
continuous_vars = [
    ESA_CCI.rename("biomass"), # double
    nearest_mature.rename("nearest_mature"), # double
    soil, # float
    terraclim.select(["mean_soil", "mean_vpd", "mean_temp", "mean_def", 
                      "mean_srad", "mean_pr", "mean_aet", "mean_pdsi"]) # float and int16
]

# Reproject categorical variables using first
categorical_vars = [
    fire, # int64
    ecoreg, # int16
    categorical, # int8
    topography # int8
]

# Create unified dataset by directly combining bands
unified_data = ee.Image.cat([
    age, # int8
    floodable_forests,
    distance_deep_forest, # int16
    sur_cover, # float
    *[reproject_mean(img) for img in continuous_vars],
    *[reproject_first(img) for img in categorical_vars],
    ee.Image.pixelLonLat().rename(['lon', 'lat'])
])

unified_data_pasture = ee.Image.cat([
    unified_data,
    pasture_area.rename('pasture_area')
])

unified_data_secondary = ee.Image.cat([
    unified_data,
    secondary_area, #double
    reproject_mean(quarters_ecoreg_biomass)
])

In [ ]:
clip_area = map.draw

# Get FeatureCollection size and calculate chunk size
grid = ee.FeatureCollection(f"{data_folder}/grid_1k_amazon_secondary")
# keep only features in clip_area
grid = grid.filter(ee.Filter.bounds(clip_area))
# keep in the grid only features with biome = 1
# grid = grid.filter(ee.Filter.eq("biome", 1))

unified_fc = age.reduceRegions(grid, ee.Reducer.first(), 30)

task = ee.batch.Export.table.toAsset(
    collection = unified_fc,
    description = "age_grid_1k",
    assetId = "projects/amazon-forest-regrowth/assets/grids/age_grid_1k"
)
task.start()



In [ ]:
def export_csv(name, image, grid_size, n_chunks = 1, lu_name = None):

    properties_to_export = image.bandNames().getInfo()
    
    # Get FeatureCollection size and calculate chunk size
    grid = ee.FeatureCollection(f"{data_folder}/grid_{grid_size}k_amazon_{name}")
    total_features = grid.size().getInfo()
    chunk_size = int(total_features * 1/n_chunks)

    if lu_name:
        name = f"{name}_{lu_name}"

    def process_chunk(chunk_index):
        start = chunk_index * chunk_size
        chunk = grid.toList(chunk_size, start)
        selected_pixels = ee.FeatureCollection(chunk)

        unified_fc = image.reduceRegions(selected_pixels, ee.Reducer.first(), 30)

        task = ee.batch.Export.table.toDrive(
            collection = unified_fc,
            description = f"grid_{grid_size}k_{name}_{chunk_index}",
            fileFormat = "CSV",
            selectors = [p for p in properties_to_export if p not in ['system:index', '.geo']]
        )
        task.start()

    for i in range(n_chunks):
        if i*chunk_size < total_features:
            process_chunk(i)

Since processing gets too heavy if we try to extract each point at a time for the entire Amazon at 1km resolution, we do it in chunks:

In [7]:
def export_csv(name, image, grid_size, n_chunks = 1, lu_name = None):

    properties_to_export = image.bandNames().getInfo()
    
    # Get FeatureCollection size and calculate chunk size
    grid = ee.FeatureCollection(f"{data_folder}/grid_{grid_size}k_amazon_{name}")
    total_features = grid.size().getInfo()
    chunk_size = int(total_features * 1/n_chunks)

    if lu_name:
        name = f"{name}_{lu_name}"

    def process_chunk(chunk_index):
        start = chunk_index * chunk_size
        chunk = grid.toList(chunk_size, start)
        selected_pixels = ee.FeatureCollection(chunk)

        unified_fc = image.reduceRegions(selected_pixels, ee.Reducer.first(), 30)

        task = ee.batch.Export.table.toDrive(
            collection = unified_fc,
            description = f"grid_{grid_size}k_{name}_{chunk_index}",
            fileFormat = "CSV",
            selectors = [p for p in properties_to_export if p not in ['system:index', '.geo']]
        )
        task.start()

    for i in range(n_chunks):
        if i*chunk_size < total_features:
            process_chunk(i)

export_csv("pastureland", unified_data_pasture, 1, n_chunks = 60)
# export_csv("secondary", unified_data_secondary, 1, n_chunks = 30)
# export_csv("secondary", unified_data_secondary, 10, n_chunks = 30)


In [54]:
suffixes = [
    "aggregated_all",
    "non_aggregated_all",
    "non_aggregated_10yr",
    "non_aggregated_5yr"
]

for suffix in suffixes:
    combined_image = ee.Image.cat([
        unified_data, 
        ee.Image(f"{data_folder}/land_use_{suffix}")
    ])
    # export_csv("secondary", combined_image, 10, n_chunks=30, lu_name=suffix)


In [16]:
def export_csv(name, image, grid_size, n_chunks = 1):

    # Get FeatureCollection size and calculate chunk size
    grid = ee.FeatureCollection(f"{data_folder}/grid_{grid_size}k_amazon_{name}")
    # keep only features in clip_area
    grid = grid.filter(ee.Filter.bounds(clip_area))
    # keep in the grid only features with biome = 1
    # grid = grid.filter(ee.Filter.eq("biome", 1))
    total_features = grid.size().getInfo()
    chunk_size = int(total_features * 1/n_chunks)

    def process_chunk(chunk_index):
        start = chunk_index * chunk_size
        chunk = grid.toList(chunk_size, start)
        selected_pixels = ee.FeatureCollection(chunk)

        unified_fc = image.reduceRegions(selected_pixels, ee.Reducer.first(), 30)

        task = ee.batch.Export.table.toAsset(
            collection = unified_fc,
            description = f"{grid_size}k_amazon_{name}",
            assetId = f"projects/amazon-forest-regrowth/assets/grids/{name}_{grid_size}k"
        )
        task.start()

    i = 0
    if i*chunk_size < total_features:
        process_chunk(i)

# export_csv("pastureland", unified_data_pasture, 1, n_chunks = 60)
export_csv("secondary", unified_data_secondary, 1, n_chunks = 2)
# export_csv("secondary", unified_data_secondary, 10, n_chunks = 2)


## Different land use aggregations


## Field Data

In [59]:
field_data = ee.FeatureCollection(f"{data_folder}/field")

# Reproject continuous variables using mean
continuous_vars = [
    nearest_mature.rename("nearest_mature"),
    soil,
    terraclim.select(["mean_soil", "mean_vpd", "mean_temp", "mean_def", 
                      "mean_srad", "mean_pr", "mean_aet", "mean_pdsi"])
]

unified_data_field = ee.Image.cat([
    floodable_forests,
    distance_deep_forest,
    sur_cover,
    *[reproject_mean(img) for img in continuous_vars],  # Exclude ESA_CCI from field data
    *[reproject_first(img) for img in categorical_vars],
    ee.Image.pixelLonLat().rename(['lon', 'lat'])
])

unified_fc = unified_data_field.reduceRegions(field_data, ee.Reducer.first(), 30)

properties_to_export = unified_data_field.bandNames().getInfo()
properties_to_export = properties_to_export + ["age", "site_id", "plot_id", "field_biom"]

# Export task to Google Drive
task = ee.batch.Export.table.toDrive(
    collection = unified_fc,
    description = 'field_predictors',
    fileFormat = "CSV",
    selectors = [p for p in properties_to_export if p not in ['system:index', '.geo']]
)
task.start()

In [ ]:
mature_biomass = ee.Image(f"{data_folder}/mature_biomass")
mature_biomass_area_2020 = ee.Image(f"{data_folder}/mature_biomass_area_2020")


# Most efficient - uses reduceRegion with bestEffort
total_biomass = mature_biomass.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mature_biomass.geometry(),
    scale=mature_biomass.projection().nominalScale(),
    maxPixels=1e9,
    bestEffort=True
).getInfo()

print(f"Total biomass: {total_biomass}")

Total biomass: {'mature_biomass': 19499281737}


In [16]:
# Load the categorical image and select the 'biome' band
biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).rename("biome_mask")
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))
mature_mask = lulc.eq(3).reduce(ee.Reducer.allNonZero()).unmask(0).updateMask(biomes_mask)

# Get area coverage in million hectares
pixel_area = ee.Image.pixelArea()
valid_pixels = mature_mask
covered_area = pixel_area.updateMask(valid_pixels).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mature_biomass.geometry(),
    scale=mature_biomass.projection().nominalScale(),
    maxPixels=1e9,
    bestEffort=True
).getInfo()

# Convert from m² to million hectares
# 1 hectare = 10,000 m²
# 1 million hectares = 10,000,000,000 m²
area_million_hectares = covered_area['area'] / 1e10

print(f"Covered area: {area_million_hectares:.2f} million hectares")

Covered area: 422.36 million hectares


In [17]:
# Load the categorical image and select the 'biome' band
biomes = ee.Image(f"{data_folder}/categorical").select("biome")
biomes_mask = biomes.eq(1).rename("biome_mask")
lulc = (ee.Image("projects/mapbiomas-public/assets/brazil/lulc/collection9/mapbiomas_collection90_integration_v1")
            .select([f"classification_{year}" for year in config.range_1985_2020])
            .byte()
            .rename([str(year) for year in config.range_1985_2020]))
mature_mask = lulc.eq(15).reduce(ee.Reducer.allNonZero()).unmask(0).updateMask(biomes_mask)

# Get area coverage in million hectares
pixel_area = ee.Image.pixelArea()
valid_pixels = mature_mask
covered_area = pixel_area.updateMask(valid_pixels).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=mature_biomass.geometry(),
    scale=mature_biomass.projection().nominalScale(),
    maxPixels=1e9,
    bestEffort=True
).getInfo()

# Convert from m² to million hectares
# 1 hectare = 10,000 m²
# 1 million hectares = 10,000,000,000 m²
area_million_hectares = covered_area['area'] / 1e10

print(f"Covered area: {area_million_hectares:.2f} million hectares")

KeyboardInterrupt: 